In [1]:
import nltk
from nltk.corpus import twitter_samples
import pandas as pd
import tensorflow as tf

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [4]:
#process_tweet(): cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets


    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
      
        if (word not in stopwords_english and   word not in string.punctuation):  
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

            
            #############################################################
            # 1 remove stopwords
            # 2 remove punctuation
            # 3 stemming word
            # 4 Add it to tweets_clean

    return tweets_clean

In [5]:

#build_freqs counts how often a word in the 'corpus' (the entire set of tweets) was associated with
# a positive label '1'         or
# a negative label '0',

#then builds the freqs dictionary, where each key is a (word,label) tuple,

#and the value is the count of its frequency within the corpus of tweets.

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)

            #############################################################
            #Update the count of pair if present, set it to 1 otherwise
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [6]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [7]:
# split the data into two pieces, one for training and one for testing
#############################################################
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [8]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [9]:
# create frequency dictionary
#############################################################
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


In [10]:
# Example
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [11]:

def extract_features(tweet, freqs):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # tokenizes, stems, and removes stopwords
    #############################################################
    output = []
    for word_l in tweet:
        word_l = process_tweet(word_l)

        # 3 elements in the form of a 1 x 3 vector
        x = np.zeros((1, 3))

        #bias term is set to 1
        x[0,0] = 1

        # loop through each word in the list of words
        for word in word_l:

            # increment the word count for the positive label 1
            x[0,1] += freqs.get((word, 1.0),0)

            # increment the word count for the negative label 0
            x[0,2] += freqs.get((word, 0.0),0)


        assert(x.shape == (1, 3))
        output.append(x)
    return output


In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [13]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(tf.convert_to_tensor(extract_features(train_x, freqs)), train_y, epochs=10)

Epoch 1/10
250/250 [==============================] - 0s 1ms/step - loss: 45.3422 - accuracy: 0.9927
Epoch 2/10
250/250 [==============================] - 0s 1ms/step - loss: 38.8909 - accuracy: 0.9930
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 32.1879 - accuracy: 0.9933
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 25.7436 - accuracy: 0.9925
Epoch 5/10
250/250 [==============================] - 0s 1ms/step - loss: 18.7308 - accuracy: 0.9926
Epoch 6/10
250/250 [==============================] - 0s 1ms/step - loss: 13.0446 - accuracy: 0.9925
Epoch 7/10
250/250 [==============================] - 0s 1ms/step - loss: 6.5628 - accuracy: 0.9915
Epoch 8/10
250/250 [==============================] - 0s 1ms/step - loss: 57.6760 - accuracy: 0.9854
Epoch 9/10
250/250 [==============================] - 0s 1ms/step - loss: 46.8045 - accuracy: 0.9919
Epoch 10/10
250/250 [==============================] - 0s 1ms/step - loss: 40.3067 - accurac

In [21]:
model.evaluate(tf.convert_to_tensor(extract_features(test_x, freqs)), test_y)

63/63 [==============================] - 0s 1ms/step - loss: 5.5888 - accuracy: 0.9945


[5.588825225830078, 0.9944999814033508]